#### Este Projeto tem como objetivo obter o conteudo de artigos da internet, armazenar seu conteudo em um banco de dados, fazer tratativas, criar nuvens de palavras, mostrar verbos e nomes relevantes , em seguida fazendo trinamentos para identificar a natureza das frases do texto, como positivas, negativas ou neutras

#### This project has the purpose of geting the content of a few internet articles, store said contents in a database, then treat it, create a wordcloud, show verbs and relevant names, then train it to identify the nature of the text's sentences, as positive, negative or neutr

In [ ]:
## importando bibliotecas
## importing Libraries

from urllib.request import urlopen
from bs4 import BeautifulSoup
import bs4
import urllib.request as urllib_request
import pandas as pd
import pymongo
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from wordcloud import WordCloud
from os import path
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from wordcloud import STOPWORDS



passos necessarios do colab

In [ ]:
## etapa necessaria para uso do codigo no colab, ignorar em outros casos
## necessary step when runnig on colaboratory, ignore otherwise

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## RODAR APENAS UMA VEZ!!# baixando arquivo spacy para npl
## RUN ONLY ONCE!! downloading spacy file for npl
!python -m spacy download pt_core_news_sm

✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [ ]:
## necessario para conexão com o atlas 
## needed to connect to atlas
!pip install dnspython

Conectando ao mongoDB

In [ ]:
## criando o banco de dados
## creating Database

cliente = pymongo.MongoClient("mongodb://localhost:27017/")
artigos_db = cliente["Artigos_Database"]
artigos_col = artigos_db["Artigos"]

Funções


In [ ]:
## função responsavel por remover residuos do codigo html e partes indesejadas de texto dos resultados do scraping
## function responsible for removing html residue and unwanted bits of text from the scrapping results

def tratativa_um (paragrafos):
  paragrafos = paragrafos.replace('\n', '').replace('\r', '').replace('\t', '').replace('<p>','').replace('</p>','').replace('      Primeira Página da Folha de', '').replace('             - Reprodução', '')
  paragrafos = paragrafos.lower()
  return paragrafos

In [ ]:
## função responsavel por remover stopwords e tokenizar o texto
## function responsible for removing stopwords and tokenizing the text

def tratativa_dois(texto):
    nlp = spacy.load("pt_core_news_sm")
    texto = nlp(texto)
    
    tokens_validos = []
    for token in texto:
      nonstop = not token.is_stop and token.is_alpha
      if nonstop:
             tokens_validos.append(token.text)      

    return " ".join(tokens_validos)


In [ ]:
## função que cria e exibe a wordcloud
## function that creates and displays the wordcloud

def word_cloud(text):
    stopwords = set(STOPWORDS)
    wc = WordCloud(width=800, height=500, background_color="black", max_words=2000,stopwords=stopwords, contour_width=3, contour_color='green').generate(text)
    
    plt.figure(figsize=(15, 10))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    plt.show()

In [ ]:
## função similar á tratativa_dois, destinada para dataframes, usada para a tratativa dos tweets
## function similar to tratativa_dois, meant for dataframes, only used to treat the tweets

def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)

    if len(tokens_validos) > 2:
        return " ".join(tokens_validos)


Trabalhando com as URLs

In [ ]:
## Gerando uma lista contendo o conteudo dos artigos, pegando os textos a partir da url, tratando e imprimindo wordclouds para cada um
## generating a list with the articles' contents, scrapping them from the urls, treating them and printing the wordclouds for each

url = [
       'https://www1.folha.uol.com.br/banco-de-dados/2021/02/1971-astronautas-recebem-autorizacao-da-nasa-para-tentar-o-pouso-na-lua.shtml',
       'https://www1.folha.uol.com.br/banco-de-dados/2021/01/1971-governo-afirma-que-troca-com-sequestradores-nao-vai-virar-rotina.shtml',
       'https://www1.folha.uol.com.br/banco-de-dados/2021/01/1971-piloto-italiano-morre-apos-ferrari-explodir-em-batida-na-argentina.shtml',
       'https://www1.folha.uol.com.br/banco-de-dados/2021/01/1971-comecam-em-janeiro-os-depositos-do-pis.shtml',
       'https://www1.folha.uol.com.br/banco-de-dados/2020/12/1970-metodo-de-exame-de-sangue-para-tentar-detectar-cancer-e-descoberto.shtml',
       'https://www1.folha.uol.com.br/banco-de-dados/2020/12/1970-parlamento-da-italia-aprova-a-lei-do-divorcio-vaticano-reage.shtml',
       'https://www1.folha.uol.com.br/banco-de-dados/2020/12/1970-condenacao-a-morte-na-urss-provoca-protestos-em-outros-paises.shtml',
       'https://www1.folha.uol.com.br/banco-de-dados/2020/12/1970-tribunal-espanhol-condena-6-a-morte-decisao-e-criticada-na-europa.shtml',
       'https://www1.folha.uol.com.br/banco-de-dados/2020/12/1971-pena-e-alterada-e-2-presos-se-livram-da-condenacao-a-morte-na-urss.shtml',
       'https://www1.folha.uol.com.br/banco-de-dados/2021/01/1971-lista-de-presos-que-serao-trocados-por-embaixador-esta-completa.shtml'
       ]

lista = []

for noticia in url:
    response = urlopen(noticia)
    html = response.read()

    soup = BeautifulSoup(html, 'html.parser')
    paragrafos = soup.find('div', class_='c-news__body').getText()
    titulo = soup.title.string
    p_um = tratativa_um(paragrafos)
    artigo = {'titulo':titulo, 'paragrafos':p_um}
    p_dois = tratativa_dois(p_um)
    print(artigo)
    print(p_dois)
    word_cloud(str(p_dois))
    lista.append([p_dois])
    




Output hidden; open in https://colab.research.google.com to view.

In [ ]:
## usando o spacy para retornar todos os verbos de todos os artigos
## using spacy to return all the verbs from the articles

nlp = spacy.load("pt_core_news_sm")
textos = nlp(str(lista))

verbos = []
for token in textos:
  if token.pos_ == 'VERB' and token.is_alpha == True and token.orth_ not in verbos:
    verbos.append(token.orth_)

verbos 

['apollo',
 'autorizados',
 'pousar',
 'dado',
 'apresentado',
 'feitas',
 'conseguir',
 'ligadas',
 'desmontaram',
 'submeteram',
 'encontrado',
 'entendo',
 'funciona',
 'declarou',
 'divulgou',
 'dizendo',
 'adotado',
 'aceitar',
 'soltar',
 'estavam',
 'viajaram',
 'chamados',
 'significa',
 'tornar',
 'sequestrado',
 'informou',
 'morreu',
 'prova',
 'bateu',
 'empurrava',
 'pegou',
 'levado',
 'sofreu',
 'deteve',
 'impediu',
 'revoltados',
 'agredissem',
 'constituído',
 'vigorar',
 'contribuirão',
 'renda',
 'creditado',
 'casar',
 'aposentar',
 'inválido',
 'utilizado',
 'anunciaram',
 'caspar',
 'publicado',
 'disseram',
 'precisam',
 'conduziu',
 'existem',
 'aprovou',
 'madrugada',
 'institui',
 'entrará',
 'sancionada',
 'giuseppe',
 'recolher',
 'submetido',
 'viajando',
 'interrompeu',
 'manifestar',
 'desgosto',
 'trará',
 'acusados',
 'decidida',
 'gerou',
 'receberam',
 'disse',
 'pedir',
 'feita',
 'reclamou',
 'recusa',
 'moscou',
 'liberar',
 'houve',
 'embaixada',

In [ ]:
## usando o spacy para exibir apenas entidades nomeadas dos arigos
## using spacy to show only named entities from the articles

entidades = []
for token in textos.ents:
  if token.orth_ not in entidades:
    entidades.append(token.orth_)
entidades

['astronautas apollo',
 'nasa',
 'lua',
 'união',
 'apollo alan shepard',
 'giovanni enrico bucher',
 'chile',
 'ignazio giunti anos',
 'buenos aires',
 'argentina',
 'ferrari',
 'beltoise',
 'pis',
 'caixa econômica federal',
 'fiwed',
 'caspar',
 'the lancet',
 'itália',
 'papa paulo',
 'austrália',
 'santa fé',
 'urss',
 'golda',
 'israel',
 'moscou',
 'inglaterra',
 'notícias tass',
 'espanha',
 'francisco franco',
 'vaticano',
 'europa',
 'república socialista federativa soviética rússia',
 'mark dymshits',
 'edward kuznetsov',
 'suíça',
 'brasil',
 'alfredo buzaid',
 'mário gibson barboza']

In [ ]:
## criando um dataframe com os textos tratados
## creating a dataframe with the treated texts

pd.set_option('display.max_column',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_seq_items',None)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)


df_news = pd.DataFrame(lista)
df_news = df_news.rename(columns={0: 'tratado'})
df_news

,tratado
0,astronautas apollo formalmente autorizados nasa a pousar lua aval dado apesar o junção módulos comando e o lunar apresentado dificuldades a união várias tentativas feitas conseguir a naves firmemente ligadas astronautas desmontaram o mecanismo acoplamento e o submeteram a testes nenhum defeito encontrado entendo absolutamente funciona perfeitamente declarou o comandante missão apollo alan shepard jr fevereiro
1,sequestro rio embaixador suíça brasil giovanni enrico bucher o governo brasileiro divulgou mensagem dizendo adotado comportamento diferente e único n caso a o aceitar soltar presos troca liberdade diplomata pessoas estavam presas viajaram o chile o atendimento exigências feitas a o governo subversivos o embarque chamados presos políticos país estrangeiro significa o processo venha a tornar rotina embaixador sequestrado subversivo solto informou o texto mensagem janeiro
2,o piloto italiano ignazio giunti anos morreu n domingo acidente prova automobilismo quilômetros buenos aires argentina bateu a ferrari o o piloto francês beltoise empurrava gasolina o choque a ferrari pegou fogo o italiano levado a o hospital sobreviveu sofreu queimaduras graves e parada polícia argentina deteve beltoise declaração formal e impediu mecânicos giunti estavam revoltados agredissem o piloto francês janeiro
3,o pis programa integração social fundo será constituído base faturamento empresas e dedução percentual imposto renda começou a vigorar n domingo empresas industriais e comerciais contribuirão vendas e imposto renda o pis n dinheiro será creditado contas individuais abertas caixa econômica federal e o trabalhador casar aposentar tornar inválido acidente trabalho o saldo utilizado a compra casa própria janeiro
4,cientistas britânicos anunciaram tenham descoberto método exame sangue conduza a pronta identificação tumores informe fiwed e caspar publicado revista científica the lancet disseram pesquisas resultados aparentemente fantásticos precisam confirmação elabore o informe investigação enfermidades nervoso central conduziu descoberta certas células sangue sensíveis s proteínas existem células cancerígenas presentes dezembro
5,votos a câmara deputados itália aprovou madrugada o projeto lei institui o divórcio lei entrará vigor seja sancionada presidente giuseppe opositores a o divórcio começaram a recolher assinaturas a o tema seja submetido a papa paulo viajando austrália interrompeu atividades manifestar o profundo desgosto o divórcio trará s famílias italianas especialmente s crianças e lesivo concordata a santa fé e o italiano dezembro
6,a condenação morte acusados tentativa sequestro avião decidida tribunal urss gerou protestos judeus soviéticos e receberam israelense golda disse o único crime d o pedir permissão a israel d manifestação feita israelenses reclamou recusa moscou liberar a saída judeus inglaterra houve protesto embaixada agência russa notícias tass chamou críticas explosão histeria antissoviética dezembro
7,o tribunal militar burgos espanha condenou morte separatistas bascos acusados banditismo e terrorismo n d receberam a pena capital decisão precedentes teria o objetivo impedir o generalíssimo francisco franco comute a noves receberam penas variam e anos prisão jovem basca vaticano e diversos países europa reagiram o veredicto pedindo a o governo espanhol conceda clemência a separatistas bascos condenados dezembro
8,o supremo tribunal apelações república socialista federativa soviética rússia comutou n moscou a pena morte imposta a judeus leningrado cumprirão anos trabalhos presos mark dymshits e edward kuznetsov acusados sequestrar pequeno avião acordo círculos diplomáticos ocidentais a extraordinária rapidez tramitação apelação mostra o governo moscou desejava evitar manifestações a condenação fossem feitas mundo janeiro
9,o governo informou recebeu a lista completa presos serão libertados troca sequestro embaixador suíça brasil giovanni enrico diplomata sequestrado dezembro rio nota oficial ministros ju

Treinamento para reconhecer a natureza de cada artigo

In [ ]:
## importando o csv com tweets para o treinamento e tratando os mesmos
## importing the tweet csv for treatment and training

df = pd.read_csv("/content/drive/MyDrive/Sprint 3/Tweets_Mg.csv", encoding='utf-8')
df_treino = df[["Text", "Classificacao"]]

nlp = spacy.load("pt_core_news_sm")
doc = nlp(str(df_treino[:5500]))

textos_para_tratamento = (titulos.lower() for titulos in df_treino["Text"])
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento, batch_size = 1000, n_process = -1)]


In [ ]:
## inserindo os textos tratados no dataframe dos tweets
## inserting treated text on tweets dataframe

df_treino.insert(1, "tratado", textos_tratados)

df_treino = df_treino.dropna()
df_treino = df_treino.drop_duplicates(subset='tratado', keep='first')
df_treino

,Text,tratado,Classificacao
0,���⛪ @ Catedral de Santo Antônio - Governador Valadares/MG https://t.co/JSbKamIqUJ,catedral santo antônio governador valadares mg,Neutro
1,"� @ Governador Valadares, Minas Gerais https://t.co/B3ThIDJCSf",governador valadares minas gerais,Neutro
4,��� PSOL vai questionar aumento de vereadores e prefeito de BH na Justiça - Politica - Estado de Minas https://t.co/DMg7BGsek5,psol questionar aumento vereadores e prefeito bh justiça politica minas,Negativo
5,""" bom é bandido morto""\nDeputado Cabo Júlio é condenado e fica inelegível por 10 anos - Politica - Estado de Minas https://t.co/3GfAqvrFHS",bandido morto deputado cabo júlio condenado e fica inelegível anos politica minas,Neutro
6,"""..E 25% dos mineiros dizem não torcer para time nenhum,mesmo dentro de um estado com Atlético-MG e Cruzeiro. Pq?.."" https://t.co/fN5evlLQsR",e mineiros torcer time nenhum e cruzeiro pq,Neutro
7,"""A gigantesca barba do mal"" em destaque no caderno Cultura do Estado de Minas. https://t.co/PxNurIkTlw",a gigantesca barba destaque caderno cultura minas,Neutro
8,"""BB e governo de Minas travam disputa sobre depósitos judiciais"" https://t.co/CnMu2A2Qo5",bb e governo minas travam disputa depósitos judiciais,Negativo
9,"""com vcs bh fica pequena!"" Belo Horizonte (pron. [bɛloɾiˈzõntʃi][10]) é a capital do estado de MG, com uma área de aproximadamente 331 km²",vcs bh fica pequena belo horizonte pron a capital mg aproximadamente,Neutro
10,"""Daí a gente visita governador valadares""\n""Que lugar eh esse mãe?""\n""MG, a gente tem uns parente la""\n\nMANO onde q eu NÃO tenho parente",d a gente visita governador valadares eh mãe mg a gente parente la mano q parente,Neutro
11,"""É bonita e é bonita..."" \n#latergram #ibituruna #home @ Governador Valadares, Minas Gerais https://t.co/y12TtNqpnH",bonita e bonita latergram ibituruna home governador valadares minas gerais,Neutro


In [ ]:
## treinando modelo
## training model

x = df_treino["tratado"].values
classes = df_treino["Classificacao"].values
vectorizer = CountVectorizer(analyzer = "word")
tweets = vectorizer.fit_transform(x)

SEED = 42
np.random.seed(SEED)

modelo = MultinomialNB()
modelo.fit(tweets, classes)



MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
## classificando os artigos
## classifying the articles

testes = df_news.tratado
freq_testes = vectorizer.transform(testes)

class_news = modelo.predict(freq_testes)


In [ ]:
## mostrando os resultados
## showing the results

df_news.insert(1, "Classificacao", class_news)
df_news

,tratado,Classificacao
0,astronautas apollo formalmente autorizados nasa a pousar lua aval dado apesar o junção módulos comando e o lunar apresentado dificuldades a união várias tentativas feitas conseguir a naves firmemente ligadas astronautas desmontaram o mecanismo acoplamento e o submeteram a testes nenhum defeito encontrado entendo absolutamente funciona perfeitamente declarou o comandante missão apollo alan shepard jr fevereiro,Neutro
1,sequestro rio embaixador suíça brasil giovanni enrico bucher o governo brasileiro divulgou mensagem dizendo adotado comportamento diferente e único n caso a o aceitar soltar presos troca liberdade diplomata pessoas estavam presas viajaram o chile o atendimento exigências feitas a o governo subversivos o embarque chamados presos políticos país estrangeiro significa o processo venha a tornar rotina embaixador sequestrado subversivo solto informou o texto mensagem janeiro,Neutro
2,o piloto italiano ignazio giunti anos morreu n domingo acidente prova automobilismo quilômetros buenos aires argentina bateu a ferrari o o piloto francês beltoise empurrava gasolina o choque a ferrari pegou fogo o italiano levado a o hospital sobreviveu sofreu queimaduras graves e parada polícia argentina deteve beltoise declaração formal e impediu mecânicos giunti estavam revoltados agredissem o piloto francês janeiro,Neutro
3,o pis programa integração social fundo será constituído base faturamento empresas e dedução percentual imposto renda começou a vigorar n domingo empresas industriais e comerciais contribuirão vendas e imposto renda o pis n dinheiro será creditado contas individuais abertas caixa econômica federal e o trabalhador casar aposentar tornar inválido acidente trabalho o saldo utilizado a compra casa própria janeiro,Neutro
4,cientistas britânicos anunciaram tenham descoberto método exame sangue conduza a pronta identificação tumores informe fiwed e caspar publicado revista científica the lancet disseram pesquisas resultados aparentemente fantásticos precisam confirmação elabore o informe investigação enfermidades nervoso central conduziu descoberta certas células sangue sensíveis s proteínas existem células cancerígenas presentes dezembro,Negativo
5,votos a câmara deputados itália aprovou madrugada o projeto lei institui o divórcio lei entrará vigor seja sancionada presidente giuseppe opositores a o divórcio começaram a recolher assinaturas a o tema seja submetido a papa paulo viajando austrália interrompeu atividades manifestar o profundo desgosto o divórcio trará s famílias italianas especialmente s crianças e lesivo concordata a santa fé e o italiano dezembro,Neutro
6,a condenação morte acusados tentativa sequestro avião decidida tribunal urss gerou protestos judeus soviéticos e receberam israelense golda disse o único crime d o pedir permissão a israel d manifestação feita israelenses reclamou recusa moscou liberar a saída judeus inglaterra houve protesto embaixada agência russa notícias tass chamou críticas explosão histeria antissoviética dezembro,Neutro
7,o tribunal militar burgos espanha condenou morte separatistas bascos acusados banditismo e terrorismo n d receberam a pena capital decisão precedentes teria o objetivo impedir o generalíssimo francisco franco comute a noves receberam penas variam e anos prisão jovem basca vaticano e diversos países europa reagiram o veredicto pedindo a o governo espanhol conceda clemência a separatistas bascos condenados dezembro,Neutro
8,o supremo tribunal apelações república socialista federativa soviética rússia comutou n moscou a pena morte imposta a judeus leningrado cumprirão anos trabalhos presos mark dymshits e edward kuznetsov acusados sequestrar pequeno avião acordo círculos diplomáticos ocidentais a extraordinária rapidez tramitação apelação mostra o governo moscou desejava evitar manifestações a condenação fossem feitas mundo janeiro,Neutro
9,o governo informou recebeu a lista completa presos serão libertados troca sequestro embaixador suíça bras

In [ ]:
## salvando os resultados no banco de dados
## saving said results in the database

dados = df_news.to_dict('list')
g_dados = ((dados))
artigos_col.insert_one(g_dados)